In [1]:
from room import Room
from player import Player
from world import World

import random
from ast import literal_eval

# Load world
world = World()


# You may uncomment the smaller graphs for development and testing purposes.
# map_file = "maps/test_line.txt"
# map_file = "maps/test_cross.txt"
# map_file = "maps/test_loop.txt"
# map_file = "maps/test_loop_fork.txt"
map_file = "maps/main_maze.txt"

# Loads the map into a dictionary
room_graph=literal_eval(open(map_file, "r").read())
world.load_graph(room_graph)

In [2]:
# Print an ASCII map
world.print_rooms()

#####
#                                                                                                                                                           #
#                                                        434       497--366--361  334--384--435  476                                                        #
#                                                         |                   |    |              |                                                         #
#                                                         |                   |    |              |                                                         #
#                                              477--443  425            386--354--321  338  313  380--445--446                                              #
#                                                    |    |              |         |    |    |    |    |                                                    #
#                                             

In [3]:
# init
player = Player(world.starting_room)
memory = dict()
travel_log = []
start_loc = player.current_room.id
memory[start_loc] = ["Start", "Gray"]
memory

{0: ['Start', 'Gray']}

In [4]:
def get_connected():
    global player
    exits = player.current_room.get_exits()
    exit_ids = [player.current_room.get_room_in_direction(direction).id for direction in player.current_room.get_exits()]
    num_exits = len(exits)
    return num_exits, exit_ids, exits

In [5]:
num_exits, exit_ids, exits = get_connected()
print(num_exits, exit_ids, exits)

4 [4, 8, 3, 1] ['n', 's', 'w', 'e']


In [6]:
# unexplored = [node for node in exit_ids if node not in memory]
# if len(unexplored) > num_exits:

# Begin Travel function

In [7]:
# num_exits, exit_ids, exits = get_connected()
# for num in range(num_exits):
#     if exit_ids[num] not in memory:
#         memory[exit_ids[num]] = [player.current_room.id, "White"]
# mem_filter = {k:v for (k,v) in memory.items() if (k in exit_ids)&(v[1]=="White")}

In [8]:
def travel(travel_log):
    # if node 0 is not black then 
    if memory[0][1] != "Black":
        # then
        # then check which nodes are connected to the current node
        num_exits, exit_ids, exits = get_connected()
        # if any nodes are unexplored
        unexplored = [node for node in exit_ids if node not in memory]
        if len(unexplored) > 0:
            # Then make the current node gray
            memory[player.current_room.id][1] = "Gray"
            # Get the first Unexplored node
            next_node = unexplored[0]
            # add the new node to memory using [node_leaving, White]
            memory[next_node] = [player.current_room.id, "White"]
            # travel to the first unexplored node in the list
            for num in range(num_exits):
                if exit_ids[num] == next_node:
                    dir_next_node = exits[num]
            player.travel(dir_next_node)
            # append the direction be travled to the travel log
            travel_log.append(dir_next_node)
            # Recursion
            return travel(travel_log)
        # else
        else:
            # Then Current node is marked black
            memory[player.current_room.id][1] = "Black"
            if memory[0][1] != "Black":
                # travel to this nodes parent node
                next_node = memory[player.current_room.id][0]
                for num in range(num_exits):
                    if exit_ids[num] == next_node:
                        dir_next_node = exits[num]
                player.travel(dir_next_node)
                # Append the direction of travel to list
                travel_log.append(dir_next_node)
                # Recursion
                return travel(travel_log)
            else:
                # print(travel_log)
                return travel_log
    # Else
    else:
        # Return travel log
        # print(travel_log)
        return travel_log

# Old function

In [9]:
# def travel(travel_log):
#     # if node 0 is not black then 
#     if memory[0][1] != "Black":
#         # then
#         # then check which nodes are connected to the current node
#         num_exits, exit_ids, exits = get_connected()
#         # if any nodes are unexplored
#         unexplored = [node for node in exit_ids if node not in memory]
#         if len(unexplored) > 0:
#             # Then make the current node gray
#             memory[player.current_room.id][1] = "Gray"
#             # Get the first Unexplored node
#             next_node = unexplored[0]
#             # add the new node to memory using [node_leaving, White]
#             memory[next_node] = [player.current_room.id, "White"]
#             # travel to the first unexplored node in the list
#             for num in range(num_exits):
#                 if exit_ids[num] == next_node:
#                     dir_next_node = exits[num]
#             player.travel(dir_next_node)
#             # append the direction be travled to the travel log
#             travel_log.append(dir_next_node)
#             # Recursion
#             travel(travel_log)
#         # else
#         else:
#             # Then Current node is marked black
#             memory[player.current_room.id][1] = "Black"
#             if memory[0][1] != "Black":
#                 # travel to this nodes parent node
#                 next_node = memory[player.current_room.id][0]
#                 for num in range(num_exits):
#                     if exit_ids[num] == next_node:
#                         dir_next_node = exits[num]
#                 player.travel(dir_next_node)
#                 # Append the direction of travel to list
#                 travel_log.append(dir_next_node)
#                 # Recursion
#                 travel(travel_log)
#             else:
#                 print(travel_log)
#                 return travel_log
#     # Else
#     else:
#         # Return travel log
#         print(travel_log)
#         return travel_log

In [10]:
traversal_path = travel(travel_log)

In [11]:
print(traversal_path)

['n', 's', 's', 'w', 'e', 'n', 'w', 'n', 'w', 'w', 's', 'n', 'w', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 'w', 'e', 'n', 'e', 's', 'n', 'e', 's', 's', 's', 'w', 'e', 'n', 'e', 'w', 'n', 'n', 'w', 'w', 'n', 'n', 'n', 'w', 's', 's', 's', 'n', 'n', 'w', 's', 's', 'w', 'e', 'n', 'w', 'e', 'n', 'e', 'n', 'e', 'n', 'n', 'w', 's', 'w', 's', 'w', 's', 'n', 'e', 'n', 'w', 'w', 's', 's', 's', 's', 's', 's', 'e', 'w', 'n', 'e', 'w', 'n', 'n', 'n', 'w', 's', 's', 's', 's', 'n', 'w', 'w', 'e', 'e', 'n', 'w', 'e', 'n', 'w', 'e', 'n', 'w', 'w', 'w', 'e', 'e', 'e', 'e', 'n', 'w', 'w', 'e', 'e', 'n', 'w', 'e', 'e', 'e', 'e', 'n', 'w', 'w', 'n', 's', 'w', 'n', 'w', 'n', 'n', 'w', 'n', 'w', 'e', 'e', 'n', 'n', 's', 'w', 'n', 's', 'w', 'e', 'e', 'e', 'e', 'e', 'n', 'w', 'w', 'e', 'e', 'e', 'n', 'n', 'w', 'n', 'w', 'e', 's', 'w', 'e', 'e', 's', 'w', 'w', 'w', 'n', 'n', 's', 'w', 'n', 'n', 's', 'w', 'w', 'w', 'e', 'e', 'e', 's', 'w', 's', 'n', 'w', 'e', 'e', 'e', 's', 'w', 'e', 'e', 'e',

In [12]:
memory

{0: ['Start', 'Black'],
 4: [0, 'Black'],
 8: [0, 'Black'],
 16: [8, 'Black'],
 3: [0, 'Black'],
 21: [3, 'Black'],
 36: [21, 'Black'],
 41: [36, 'Black'],
 76: [41, 'Black'],
 69: [41, 'Black'],
 72: [69, 'Black'],
 89: [72, 'Black'],
 104: [89, 'Black'],
 126: [104, 'Black'],
 135: [126, 'Black'],
 149: [135, 'Black'],
 191: [149, 'Black'],
 193: [191, 'Black'],
 203: [193, 'Black'],
 269: [203, 'Black'],
 315: [269, 'Black'],
 406: [315, 'Black'],
 410: [406, 'Black'],
 335: [315, 'Black'],
 346: [335, 'Black'],
 378: [335, 'Black'],
 466: [378, 'Black'],
 472: [466, 'Black'],
 481: [472, 'Black'],
 485: [481, 'Black'],
 495: [472, 'Black'],
 241: [193, 'Black'],
 256: [241, 'Black'],
 279: [256, 'Black'],
 323: [279, 'Black'],
 327: [256, 'Black'],
 362: [327, 'Black'],
 395: [362, 'Black'],
 423: [395, 'Black'],
 469: [362, 'Black'],
 156: [149, 'Black'],
 209: [156, 'Black'],
 213: [209, 'Black'],
 217: [213, 'Black'],
 271: [217, 'Black'],
 310: [271, 'Black'],
 216: [213, 'Blac

In [13]:
#! ==================== DO NOT MODIFY ====================
# TRAVERSAL TEST - DO NOT MODIFY
visited_rooms = set()
player.current_room = world.starting_room
visited_rooms.add(player.current_room)

for move in traversal_path:
    player.travel(move)
    visited_rooms.add(player.current_room)

if len(visited_rooms) == len(room_graph):
    print(f"TESTS PASSED: {len(traversal_path)} moves, {len(visited_rooms)} rooms visited")
else:
    print("TESTS FAILED: INCOMPLETE TRAVERSAL")
    print(f"{len(room_graph) - len(visited_rooms)} unvisited rooms")
#! ==================== DO NOT MODIFY ====================

TESTS PASSED: 998 moves, 500 rooms visited
